In [2]:
import pandas as pd
import numpy as np
import re
import sys
sys.path.append('/home/descombe/Desktop/photogrammetry-tools/src/utils.py')
from utils import get_sfm_data_as_dict

In [3]:
#file = '/media/gargantua/1000-plane/0000-sfm/2021/0102/20210102-163659-20200224-00001-suisse-sion-0001-loop-1-2-normalized/geodesy_test/test.csv'
#file = '/media/gargantua/1000-plane/0000-sfm/2021/0120/20210120-195740-20200225-00001-suisse-sion-0001-loop-13-normalized/geodesy/test.csv'
#file = '/media/scanvan/model/camera_40008603-40009302/20201006/20201006-174611_20200224-152852_sion-loop-7/geodesy/test.csv'
file = '/media/gargantua/1000-plane/0000-sfm/2021/0102/20210102-174708-20200224-00001-suisse-sion-0001-loop-3-normalized/geodesy/loop3.csv'
wkt = pd.read_csv(file, dtype=str)


In [4]:
wkt.head(5)

,WKT,X,Y,Z,filename
0,POINT Z (2593844.53757659 1120435.04500741 0),2593851.801105999853462,1120427.781478000106290,526.671062000000006,20200224-112105-962881.jpg
1,POINT Z (2593843.16036059 1120434.50652241 0),2593850.423890000209212,1120427.242992999963462,526.652822000000015,20200224-112106-212819.jpg
2,POINT Z (2593841.80106059 1120433.97736341 0),2593849.064590000081807,1120426.713834000052884,526.627574999999979,20200224-112106-462841.jpg
3,POINT Z (2593840.43369459 1120433.47223441 0),2593847.697224000003189,1120426.208705000113696,526.609459000000015,20200224-112106-712813.jpg
4,POINT Z (2593839.01347159 1120432.97837641 0),2593846.277001000009477,1120425.714847000082955,526.589748999999983,20200224-112106-962861.jpg


In [5]:
structure_dict, views_dict, extrinsics_dict, intrinsics = get_sfm_data_as_dict(file.replace('geodesy/loop3.csv','omvg/sfm_data_bin.json'))

In [14]:
len(extrinsics_dict)

699

In [15]:
file_out = file.replace('loop3.csv','homologous.dat')
with open(file_out, 'w') as f:
    for row_id in range(0, len(wkt), 20):
        if extrinsics_dict.get(row_id) is not None:
            #filename = wkt.iloc[row_id]['filename']
            #filename_ = views_dict.get(row_id)['ptr_wrapper']['data']['filename']
            coor_before = extrinsics_dict.get(row_id)['center']
            coor_after = wkt.iloc[row_id]['WKT']
            coor_digits = re.sub("[^0-9]", " ", coor_after).strip()
            x_after = '.'.join(coor_digits.split()[0:2])
            y_after = '.'.join(coor_digits.split()[2:4])
            z_after = wkt.iloc[row_id]['Z']
            #z_after = np.float('.'.join(coor_digits.split()[4:]))
            #x_after = wkt.iloc[row_id]['X']
            #y_after = wkt.iloc[row_id]['Y']
            for coor in coor_before:
                f.write("%s " % coor)
            f.write("%s %s %f\n" % (x_after, y_after, float(z_after)))


In [26]:
csv = pd.read_csv('/media/scanvan/model/camera_40008603-40009302/20201006/20201006-174611_20200224-152852_sion-loop-7/geodesy/_20201006-174611_20200224-152852_sion-loop-7-vodom-omvg-epsg2056.csv', dtype=str)

In [27]:
header = 'ply\nformat %s 1.0\nelement vertex %d\nproperty float64 x\nproperty float64 y\nproperty float64 z\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nend_header\n' %('ascii', len(csv))

In [15]:
csv.X

0       2593686.694522
1       2593686.581918
2       2593686.470146
3       2593686.356271
4       2593686.233954
             ...      
1729    2593945.570364
1730    2593947.212720
1731    2593948.865674
1732    2593950.534753
1733    2593952.212812
Name: X, Length: 1734, dtype: object

In [28]:

with open('/media/scanvan/model/camera_40008603-40009302/20201006/20201006-174611_20200224-152852_sion-loop-7/geodesy/_20201006-174611_20200224-152852_sion-loop-7-vodom-omvg-epsg2056.ply', 'w') as f:
    for line in header:
        f.write("%s" % line)
    for i, row in csv.iterrows():
        f.write("%s %s %s 0 255 0\n" % (row['X'], row['Y'], float(row['Z'])))
        